In [1]:
import pandas as pd
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import chardet    
rawdata = open('source.csv', 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)

Big5


In [3]:
df = pd.read_csv('source.csv',encoding=r'Big5')

In [4]:
df['發生時段']

0       08~10
1       00~02
2       00~02
3       06~08
4       10~12
        ...  
3267    09~11
3268    12~14
3269    06~08
3270    15~17
3271    18~20
Name: 發生時段, Length: 3272, dtype: object

In [5]:
time_zone = []
for r in df['發生時段']:
    if(r == '00~02' or r == '03~05' or r == '04~06' or r == '05~07'):
        time_zone.append('midnight')
    elif(r == '06~08' or r =='08~10' or r == '09~11' or r == '10~12' or r == '11~13'):
        time_zone.append('morning')
    elif(r == '12~14' or r == '14~16' or r == '15~17' or r == '15~18' or r == '16~18' or r == '17~19'):
        time_zone.append('afternoon')
    else:
        time_zone.append('night')


In [6]:
df['time_zone'] = time_zone

In [7]:
df.loc[df['time_zone']=='morning'].to_csv('morning.csv',encoding='utf-8-sig')

In [8]:
list = []

In [9]:
for r in df['發生地點']:
    list.append(r[0:6])
df['locate'] = list

In [45]:
train, test = train_test_split(df, test_size = 0.2)

In [11]:
BOW_vectorizer = CountVectorizer() 

In [12]:
BOW_vectorizer.fit(train['locate'])
train_data_BOW_features = BOW_vectorizer.transform(train['locate'])
test_data_BOW_features = BOW_vectorizer.transform(test['locate'])

In [13]:
train_data_BOW_features

<2617x12 sparse matrix of type '<class 'numpy.int64'>'
	with 2617 stored elements in Compressed Sparse Row format>

In [14]:
feature_names = BOW_vectorizer.get_feature_names()
feature_names

['臺北市中山區',
 '臺北市中正區',
 '臺北市信義區',
 '臺北市內湖區',
 '臺北市北投區',
 '臺北市南港區',
 '臺北市士林區',
 '臺北市大同區',
 '臺北市大安區',
 '臺北市文山區',
 '臺北市松山區',
 '臺北市萬華區']

In [15]:
import nltk
import jieba
# build analyzers (bag-of-words)
BOW = CountVectorizer(tokenizer=jieba.cut) 

# apply analyzer to training data
BOW.fit(train['發生地點'])

train_data_BOW_features = BOW.transform(train['time_zone'])

## check dimension
train_data_BOW_features.shape

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.606 seconds.
Prefix dict has been built successfully.


(2617, 1405)

In [52]:
feature_names = BOW.get_feature_names()


In [17]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW.transform(train['locate'])
y_train = train['time_zone']

X_test = BOW.transform(test['locate'])
y_test = test['time_zone']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (2617, 1405)
y_train.shape:  (2617,)
X_test.shape:  (655, 1405)
y_test.shape:  (655,)


In [18]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:20]

array(['morning', 'night', 'night', 'night', 'night', 'night', 'morning',
       'night', 'night', 'night', 'night', 'night', 'afternoon', 'night',
       'night', 'night', 'afternoon', 'night', 'night', 'morning'],
      dtype=object)

In [19]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

training accuracy: 0.34
testing accuracy: 0.29


In [20]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['afternoon' 'midnight' 'morning' 'night']

## Before convert
y_train[0:4]:
 1530       night
1437       night
1012     morning
1720    midnight
Name: time_zone, dtype: object

y_train.shape:  (2617,)
y_test.shape:  (655,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

y_train.shape:  (2617, 4)
y_test.shape:  (655, 4)


In [21]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  1405
output_shape:  4


In [22]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1405)]            0         
_________________________________________________________________
dense (Dense)                (None, 64)                89984     
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
_________________________________________________________________
softmax (Softmax)            (None, 4)                 0     

In [35]:
from keras.callbacks import CSVLogger

# training setting
epochs = 25
batch_size = 128

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data = (X_test, y_test))
print('training finish')

Epoch 1/25
21/21 [==============================] - 0s 4ms/step - loss: 1.3267 - accuracy: 0.3412 - val_loss: 1.3612 - val_accuracy: 0.2947
Epoch 2/25
21/21 [==============================] - 0s 4ms/step - loss: 1.3275 - accuracy: 0.3408 - val_loss: 1.3616 - val_accuracy: 0.2947
Epoch 3/25
21/21 [==============================] - 0s 4ms/step - loss: 1.3269 - accuracy: 0.3389 - val_loss: 1.3610 - val_accuracy: 0.2885
Epoch 4/25
21/21 [==============================] - 0s 3ms/step - loss: 1.3271 - accuracy: 0.3420 - val_loss: 1.3608 - val_accuracy: 0.2885
Epoch 5/25
21/21 [==============================] - 0s 2ms/step - loss: 1.3269 - accuracy: 0.3420 - val_loss: 1.3615 - val_accuracy: 0.2885
Epoch 6/25
21/21 [==============================] - 0s 2ms/step - loss: 1.3270 - accuracy: 0.3412 - val_loss: 1.3613 - val_accuracy: 0.2855
Epoch 7/25
21/21 [==============================] - 0s 4ms/step - loss: 1.3274 - accuracy: 0.3363 - val_loss: 1.3640 - val_accuracy: 0.2885
Epoch 8/25
21/21 [==

In [36]:
## predict
pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]

array([[0.27878603, 0.0523066 , 0.3376779 , 0.33122948],
       [0.32732227, 0.10465295, 0.22233465, 0.34569013],
       [0.21700029, 0.19192323, 0.25186312, 0.33921337],
       [0.24791259, 0.13043502, 0.28587985, 0.33577254],
       [0.24791259, 0.13043502, 0.28587985, 0.33577254]], dtype=float32)

In [37]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['morning', 'night', 'night', 'night', 'night'], dtype=object)

In [38]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

testing accuracy: 0.29


---KNN---


In [66]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [67]:
train_data , test_data , train_label , test_label = train_test_split(df['locate'],df['time_zone'],test_size=0.2)

In [68]:
train_data = BOW.transform(train_data)
test_data = BOW.transform(test_data)
train_data.shape

(2617, 1405)

In [69]:
test_data.shape

(655, 1405)

In [70]:
knn = KNeighborsClassifier()
knn.fit(train_data,train_label)

KNeighborsClassifier()

In [71]:
knn.score(test_data,test_label)

0.28091603053435116

---如何預測---

In [27]:
test = BOW.transform(['乾你媽的'])



In [28]:
DT_model.predict(test)[0]

'night'

In [29]:
target = df['發生地點']

In [30]:
target_bow = BOW.transform(target)

In [31]:
predict = DT_model.predict(target_bow)

In [32]:
predict = pd.Series(predict)

In [33]:
res = pd.concat([target,predict], axis=1, ignore_index=True)

In [34]:
res.to_csv('result.csv', index=False, encoding='utf-8-sig')